# Using the Text-to-Text Transfer Transformer or T5 model
Unlike traditional models that use task-specific architectures, T5 treats all natural language processing problems as text-to-text transformations, where both input and output are formatted as text sequences.

In [1]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# If CUDA is available, print additional info
if torch.cuda.is_available():
    print("CUDA device name:", torch.cuda.get_device_name(0))
    print("CUDA device count:", torch.cuda.device_count())

CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3070 Laptop GPU
CUDA device count: 1


In [14]:
from datasets import load_dataset

# Load the data
data = load_dataset("rotten_tomatoes")
data

# we will use the train split when we train a model and the test split for validating
# the results.

# additional validation split can be used to further validate
# generalization if you used the train and test splits to perform hyperparameter tuning.

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [15]:
from transformers import pipeline



# Load our model
pipe = pipeline(
"text2text-generation", # task
model="google/flan-t5-small", # model name
device="cuda:0"
)

loading configuration file config.json from cache at /home/proxi/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/0fc9ddf78a1e988dac52e2dac162b0ede4fd74ab/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_

Compared to our task-specific model, we cannot just give the model some text and
hope it will output the sentiment. Instead, we will have to instruct the model to do so.

Thus, we prefix each document with the prompt “Is the following sentence positive or
negative?”:

In [16]:
# Prepare our data
prompt = "Is the following sentence positive or negative? " # this is the prompt we will use to ask the model to classify the text
data = data.map(lambda example: {"t5": prompt + example['text']}) # add the prompt to the text
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [17]:
from tqdm import tqdm
import numpy as np
from transformers.pipelines.pt_utils import KeyDataset

# Run inference 
y_pred = []  # Initialize empty list to store model predictions (0 for negative, 1 for positive)
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):  # Process test data through pipeline with progress bar
    text = output[0]["generated_text"]  # Extract generated text from model output (first element in output list)
    y_pred.append(0 if text == "negative" else 1)  # Convert text label to binary (0=negative, 1=positive/neutral) and store

100%|██████████| 1066/1066 [00:30<00:00, 34.46it/s]


In [18]:
from sklearn.metrics import classification_report  # Import performance metrics module

def evaluate_performance(y_true, y_pred):  # Define evaluation function taking true labels and predictions
    performance = classification_report(  # Generate classification report
        y_true, y_pred,  # Compare ground truth vs predicted labels
        target_names=["Negative Revi    ew", "Positive Review"]  # Label classes for readable output
    )
    print(performance)  # Print precision, recall, f1-score metrics
# Evaluate the model performance
evaluate_performance(data["test"]["label"], y_pred)  # Call evaluation function with true labels and predictions

                     precision    recall  f1-score   support

Negative Revi    ew       0.83      0.85      0.84       533
    Positive Review       0.85      0.83      0.84       533

           accuracy                           0.84      1066
          macro avg       0.84      0.84      0.84      1066
       weighted avg       0.84      0.84      0.84      1066

